In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import itertools
from scipy import stats

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils import resample

import seaborn as seabornInstance
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics

from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras import optimizers

Using TensorFlow backend.


In [2]:
def plot_loss_accuracy(history):
    historydf = pd.DataFrame(history.history, index=history.epoch)
    plt.figure(figsize=(8, 6))
    historydf.plot(ylim=(0, max(1, historydf.values.max())))
    loss = history.history['loss'][-1]
    acc = history.history['acc'][-1]
    plt.title('Loss: %.3f, Accuracy: %.3f' % (loss, acc))

In [3]:
def plot_bin_loss_accuracy(history):
    historydf = pd.DataFrame(history.history, index=history.epoch)
    plt.figure(figsize=(8, 6))
    historydf.plot(ylim=(0, max(1, historydf.values.max())))
    loss = history.history['loss'][-1]
    acc = history.history['binary_accuracy'][-1]
    plt.title('Loss: %.3f, Accuracy: %.3f' % (loss, acc))

In [4]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [5]:
data = pd.read_csv('C:/Users/ak19919/Downloads/ml_root/kaggle/titanic/train.csv')
data.columns = data.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
data.head(10)

,passengerid,survived,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [6]:
data = data.set_index('passengerid')
data['designation'] = ((data['name'].str.rsplit(',').str[-1]).str.strip()).str.split('.').str[0]
data.nunique()

survived         2
pclass           3
name           891
sex              2
age             88
sibsp            7
parch            7
ticket         681
fare           248
cabin          147
embarked         3
designation     17
dtype: int64

In [7]:
data.shape

(891, 12)

In [8]:
data.isnull().sum()

survived         0
pclass           0
name             0
sex              0
age            177
sibsp            0
parch            0
ticket           0
fare             0
cabin          687
embarked         2
designation      0
dtype: int64

In [9]:
# Imputing missing values and adding info column

for column in ['embarked']:
    data[column].fillna(data[column].mode()[0], inplace = True)

In [10]:
def impute_median(series):
    return series.fillna(series.median())

In [11]:
salutations = {'Mme':'Miss', 'Miss':'Miss', 'Mlle':'Miss', 'Dona':'Mrs', 'Mr':'Mr', 'Mrs':'Mrs', 'Don':'Mr',
              'Rev':'Rev', 'Dr':'Dr', 'Major':'Mr', 'Lady':'Mrs', 'Sir':'Mr', 'Master':'Master', 'Ms':'Ms', 'Col':'Col',
              'Capt':'Mr', 'the Countess':'Mrs', 'Jonkheer':'Mr'}

In [12]:
data['designation'] = data['designation'].map(salutations)

In [13]:
data.head(20)

,survived,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,designation
passengerid,,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Mr
6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q,Mr
7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S,Mr
8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S,Master
9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S,Mrs


# Age prediction using Linear regression

In [14]:
data = data.drop(['name', 'cabin', 'ticket'], axis = 1)

In [15]:
data.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,designation
passengerid,,,,,,,,,
1,0,3,male,22.0,1,0,7.2500,S,Mr
2,1,1,female,38.0,1,0,71.2833,C,Mrs
3,1,3,female,26.0,0,0,7.9250,S,Miss
4,1,1,female,35.0,1,0,53.1000,S,Mrs
5,0,3,male,35.0,0,0,8.0500,S,Mr


In [16]:
# One hot encoding categorical features

cat_features = ['sex', 'embarked', 'designation']
df_cat = pd.get_dummies(data[cat_features])
data = data.drop(cat_features, axis=1)
data = pd.concat([data, df_cat], axis = 1)

In [17]:
data.head()

,survived,pclass,age,sibsp,parch,fare,sex_female,sex_male,embarked_C,embarked_Q,embarked_S,designation_Col,designation_Dr,designation_Master,designation_Miss,designation_Mr,designation_Mrs,designation_Ms,designation_Rev
passengerid,,,,,,,,,,,,,,,,,,,
1,0,3,22.0,1,0,7.2500,0,1,0,0,1,0,0,0,0,1,0,0,0
2,1,1,38.0,1,0,71.2833,1,0,1,0,0,0,0,0,0,0,1,0,0
3,1,3,26.0,0,0,7.9250,1,0,0,0,1,0,0,0,1,0,0,0,0
4,1,1,35.0,1,0,53.1000,1,0,0,0,1,0,0,0,0,0,1,0,0
5,0,3,35.0,0,0,8.0500,0,1,0,0,1,0,0,0,0,1,0,0,0


In [18]:
# Scaling the features within range (0, 1)

ss = StandardScaler()
scale_features = ['fare', 'parch', 'sibsp', 'pclass']
data[scale_features] = ss.fit_transform(data[scale_features])
#data.head()

In [19]:
quantile_list = [0, .25, .5, .75, 1.]
age_quantiles = data['age'].quantile(quantile_list)
age_quantiles

0.00     0.420
0.25    20.125
0.50    28.000
0.75    38.000
1.00    80.000
Name: age, dtype: float64

In [20]:
quantile_labels = ['0-25Q', '25-50Q', '50-75Q', '75-100Q']
# data['age_quantile_range'] = pd.qcut(data['age'], q=quantile_list)
data['age_range'] = pd.qcut(data['age'], q=quantile_list, labels=quantile_labels)
data = data.drop('age', axis=1)

In [21]:
test_age = data[data['age_range'].isnull()]
data = data.dropna(axis = 0, how ='any')

In [22]:
# Performing Train Test Split (70-30 split)

X = data.drop('age_range', axis=1).values
y = data['age_range'].values

In [25]:
X

array([[ 0.        ,  0.82737724,  0.43279337, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.        , -1.56610693,  0.43279337, ...,  1.        ,
         0.        ,  0.        ],
       [ 1.        ,  0.82737724, -0.4745452 , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 1.        , -1.56610693, -0.4745452 , ...,  0.        ,
         0.        ,  0.        ],
       [ 1.        , -1.56610693, -0.4745452 , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.82737724, -0.4745452 , ...,  0.        ,
         0.        ,  0.        ]])

In [26]:
y

[25-50Q, 50-75Q, 25-50Q, 50-75Q, 50-75Q, ..., 75-100Q, 25-50Q, 0-25Q, 25-50Q, 50-75Q]
Length: 714
Categories (4, object): [0-25Q < 25-50Q < 50-75Q < 75-100Q]

In [ ]:
data.head()

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

TypeError: take_nd() got an unexpected keyword argument 'axis'

In [ ]:
gbc = GradientBoostingClassifier()
gbc.fit(X_train, y_train)
y_pred = gbc.predict(X_test)
acc_gbc = round(accuracy_score(y_pred, y_test) * 100, 2)
print(acc_gbc)

In [ ]:
regressor = LinearRegression()  
regressor.fit(X_train, y_train)

In [ ]:
y_pred = regressor.predict(X_test)

In [ ]:
df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
df1 = df.head(25)

In [ ]:
df1

In [ ]:
kf = KFold(n_splits=5) # Define the split - into 2 folds 
kf.get_n_splits(X) # returns the number of splitting iterations in the cross-validator
print(kf)

In [ ]:
for train_index, test_index in kf.split(X):
    print('TRAIN:', train_index, 'TEST:', test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

In [ ]:
scores = cross_val_score(regressor, X, y, cv=8)
print('Cross-validated scores:', scores)

In [ ]:
# Make cross validated predictions
predictions = cross_val_predict(regressor, X, y, cv=6)
plt.scatter(y, predictions)

In [ ]:
accuracy = metrics.r2_score(y, predictions)
print('Cross-Predicted Accuracy:', accuracy)

In [ ]:
ageGrp = data.groupby(['sex','pclass','designation'])
data.age = ageGrp.age.transform(impute_median)

cabins = {"A":1, "B":2, "C":3, "D":4, "E":5, "F":6, "G":7, "N":0, "T":8}
data['cabin'] = data['cabin'].fillna('NA')
data['cabinclass'] = data['cabin'].astype(str).str[0]
data['cabinclass'] = data['cabinclass'].map(cabins)
#data.cabinclass = cabinGrp.cabinclass.transform(impute_median)
data['cabinclass'] = data['cabinclass'].astype(int)
data = data.drop(['name', 'cabin', 'ticket'], axis = 1)

In [ ]:
quantile_list = [0, .25, .5, .75, 1.]
age_quantiles = data['age'].quantile(quantile_list)
age_quantiles

In [ ]:
fig, ax = plt.subplots()
data['age'].hist(bins=30, color='#A9C5D3', 
                             edgecolor='black', grid=False)
for quantile in age_quantiles:
    qvl = plt.axvline(quantile, color='r')
ax.legend([qvl], ['Quantiles'], fontsize=10)
ax.set_title('Age Histogram with Quantiles', fontsize=12)
ax.set_xlabel('Age', fontsize=12)
ax.set_ylabel('Frequency', fontsize=12)

In [ ]:
quantile_labels = ['0-25Q', '25-50Q', '50-75Q', '75-100Q']
# data['age_quantile_range'] = pd.qcut(data['age'], q=quantile_list)
data['age_range'] = pd.qcut(data['age'], q=quantile_list, labels=quantile_labels)
data = data.drop('age', axis=1)
data.head()

In [ ]:
data.isnull().sum()

In [ ]:
df_num = data.select_dtypes(include = ['float64', 'int64'])
df_num.head()

In [ ]:
df_num.hist(figsize=(16, 20), bins=50, xlabelsize=8, ylabelsize=8)

In [ ]:
sns.heatmap(df_num.corr()[['survived']], annot=True, vmin=-1, vmax=1)

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))         # Sample figsize in inches
sns.heatmap(df_num.corr(), annot=True, square=True, vmin=-1, vmax=1)

In [ ]:
# One hot encoding categorical features

cat_features = ['sex', 'embarked', 'designation', 'age_range']
df_cat = pd.get_dummies(data[cat_features])
data = data.drop(cat_features, axis=1)
data = pd.concat([data, df_cat], axis = 1)

In [ ]:
# Scaling the features within range (0, 1)

ss = StandardScaler()
scale_features = ['fare', 'parch', 'sibsp', 'pclass']
data[scale_features] = ss.fit_transform(data[scale_features])
#data.head()

In [ ]:
# Performing Train Test Split (70-30 split)

#data = data.drop('recruitment_channel', axis=1)
X = data.drop('survived', axis=1).values
y = data['survived'].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 0)

In [ ]:
ADAMAX = optimizers.Adamax(lr = 0.002, beta_1 = 0.9, beta_2 = 0.999)
# Accuracy with ADAMAX at 0.944, loss at 0.153, F1 Score at 0.5222337125129266

ADAM = optimizers.Adam(lr = 0.01)
# Accuracy with ADAM at 0.945, loss at 0.146, F1 Score at 0.517427589592538

ADADELTA = optimizers.Adadelta(lr = 1.0, rho = 0.95)
# Accuracy with ADADELTA at , loss at 0.155, F1 Score at 0.506652474720596

ANN_model = Sequential()
ANN_model.add(Dense(64, input_shape = (X_train.shape[1],), activation = 'tanh'))
ANN_model.add(Dense(256, activation = 'tanh'))
ANN_model.add(Dense(256, activation = 'tanh'))
ANN_model.add(Dense(256, activation = 'tanh'))
ANN_model.add(Dense(128, activation = 'tanh'))
# Last layer to use sigmoid activation function (coz binary classification)
ANN_model.add(Dense(1, activation = 'sigmoid'))

ANN_model.compile(optimizer = ADAMAX, loss = 'binary_crossentropy', metrics = ['accuracy'])

ANN_history = ANN_model.fit(X_train, y_train, verbose = 0, epochs = 2000)
plot_loss_accuracy(ANN_history)

In [ ]:
# Plot Confusion matrix

Y_Pred = ANN_model.predict(X_test)
Cnf_matrix = confusion_matrix(y_test, Y_Pred.round())
np.set_printoptions(precision = 2)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(Cnf_matrix, classes=['Not Survived','Survived'],
                      title='Confusion matrix, without normalization')

In [ ]:
#extracting true_positives, false_positives, true_negatives, false_negatives

tn, fp, fn, tp = confusion_matrix(y_test, Y_Pred.round()).ravel()
print("True Negatives: ",tn)
print("False Positives: ",fp)
print("False Negatives: ",fn)
print("True Positives: ",tp)

In [ ]:
#Accuracy
Accuracy = (tn+tp)*100/(tp+tn+fp+fn) 
print("Accuracy: {:0.2f}%".format(Accuracy))
print("Accuracy Score: {}".format(accuracy_score(y_test, Y_Pred.round(), normalize = False)))

#Precision 
Precision = tp/(tp+fp) 
print("Precision: {:0.2f}".format(Precision))
print("Precision Score: {}".format(precision_score(y_test, Y_Pred.round(), pos_label = 1, average = 'binary')))

#Recall 
Recall = tp/(tp+fn) 
print("Recall: {:0.2f}".format(Recall))
print("Recall Score: {}".format(recall_score(y_test, Y_Pred.round(), pos_label = 1, average = 'binary')))

#F1 Score
f1 = (2*Precision*Recall)/(Precision + Recall)
print("F1 Score {:0.2f}".format(f1))
print("F1 Score: {}".format(f1_score(y_test, Y_Pred.round(), pos_label = 1, average = 'binary')))

#Specificity 
Specificity = tn/(tn+fp)
print("Specificity: {:0.2f}".format(Specificity))

In [ ]:
test_data = pd.read_csv('C:/Users/ak19919/Downloads/ml_root/kaggle/titanic/test.csv')
test_data.columns = test_data.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')

In [ ]:
test_data.isnull().sum()

In [ ]:
test_data['designation'] = ((test_data['name'].str.rsplit(',').str[-1]).str.strip()).str.split('.').str[0]
ageGrp = test_data.groupby(['sex','pclass','designation'])
test_data['designation'] = test_data['designation'].map(salutations)
test_data.age = ageGrp.age.transform(impute_median)

cabins = {"A":1, "B":2, "C":3, "D":4, "E":5, "F":6, "G":7, "N":0, "T":8}
test_data['cabin'] = test_data['cabin'].fillna('NA')
test_data['cabinclass'] = test_data['cabin'].astype(str).str[0]
test_data['cabinclass'] = test_data['cabinclass'].map(cabins)
#data.cabinclass = cabinGrp.cabinclass.transform(impute_median)
test_data['cabinclass'] = test_data['cabinclass'].astype(int)
test_data = test_data.drop(['name', 'cabin', 'ticket'], axis = 1)

In [ ]:
test_data.head()

In [ ]:
test_data['age_range'] = pd.qcut(test_data['age'], q=quantile_list, labels=quantile_labels)
test_data = test_data.drop('age', axis=1)
df_cat_T = pd.get_dummies(test_data[cat_features])
test_data = test_data.drop(cat_features, axis = 1)
test_data = pd.concat([test_data, df_cat_T], axis = 1)

In [ ]:
# Scaling the features within range (0, 1)
test_data[scale_features] = ss.fit_transform(test_data[scale_features])
test_data.head()

In [ ]:
test_no_id = test_data.drop('passengerid', axis = 1)
test_predictions = ANN_model.predict(test_no_id)
passengerid = test_data['passengerid']
submission_df_1 = pd.DataFrame({
                  "passengerid": passengerid, 
                  "survived": test_predictions.ravel()})

In [ ]:
submission_df_1.survived = submission_df_1.survived.round()
submission_df_1.to_csv('20200503_submission.csv',index = False)

### Accuracy with Neural Network: 0.77
## Implementing Gaussian Bayes Algorithm

In [ ]:
from sklearn.naive_bayes import GaussianNB
#Create a Gaussian Classifier
gnb = GaussianNB()
gnb.fit(X_train, y_train)

y_pred = gnb.predict(X_test)

In [ ]:
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [ ]:
test_no_id = test_data.drop('passengerid', axis = 1)
test_no_id = test_no_id.fillna(test_no_id.mean())
test_predictions = gnb.predict(test_no_id)
passengerid = test_data['passengerid']
submission_df_1 = pd.DataFrame({
                  "passengerid": passengerid, 
                  "survived": test_predictions.ravel()})

In [ ]:
submission_df_1.survived = submission_df_1.survived.round()
submission_df_1.to_csv('20200503_submission_GB.csv',index = False)

### Accuracy with Gaussian Bayes: 0.75
# Implementing Decision Tree Classifier

In [ ]:
clf = DecisionTreeClassifier()

# Train Decision Tree Classifer
clf = clf.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

In [ ]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))